In [2]:
from credentials import *
import tweepy

In [69]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tweepy import *

In [1]:
import pandas as pd
import csv
import re 
import string
import preprocessor as p

In [5]:
consumer_key = "4q1OMAPiHsnbUlWx12wCdgJ3s"
consumer_secret = "6UUTbNiSRnSIhOJCWwcZIXBdjwoCcu7QJhFoyD01gVOcxGCXrD"
access_key= "1382243111154307073-kTiHV7wKZx0eT1spoEMhO8PDlQ6DOz"
access_secret = "NTccRoqk6brfmV1mjwJj0HC3XtOJEiRNVynWrakuNTD6Q"

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

In [7]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [23]:
csvFile = open('Location Tweets Dataset', 'a')
csvWriter = csv.writer(csvFile)

In [9]:
locations=["Paris", "London", "Maldives", "Goa", "Syria", "Iran", "Iraq", "Kashmir", "Delhi", "Mumbai", "Greece", "New York"," NYC"]

In [ ]:
for word in locations:
    search_words = "#{}".format(word)
    print(search_words)

    new_search = search_words + " -filter:retweets"

    for tweet in tweepy.Cursor(api.search_tweets,q=new_search,count=100,
                               lang="en",
                               since_id=0).items():
        csvWriter.writerow([word, tweet.created_at, tweet.text.encode('utf-8'),tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8')])


#Paris


Rate limit reached. Sleeping for: 446


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from gensim.models.phrases import Phrases, Phraser
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

C:\Users\harsh\anaconda3\envs\ml\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
import torch
from transformers import AutoModel, AutoTokenizer 

In [11]:
import multiprocessing
from time import time
import numpy as np
import hashlib

In [3]:
df=pd.read_csv('Location_Tweets_Dataset.csv')
#df.columns=['Location-Search', 'Date', 'Tweet', 'Username', 'Location-Tweet']

In [36]:
df_orig=pd.read_csv('Location_Tweets_Dataset.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121248 entries, 0 to 121247
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Location-Search  121248 non-null  object
 1   Date             121248 non-null  object
 2   Tweet            121248 non-null  object
 3   Username         121229 non-null  object
dtypes: object(4)
memory usage: 3.7+ MB


In [5]:
for i in df['Tweet']:
    i.replace('b','')

In [6]:
df.head()

,Location-Search,Date,Tweet,Username
0,New York,2022-02-08 08:09:30+00:00,opportunities for mri technologist prn working...,flyjofinder
1,New York,2022-02-08 08:09:16+00:00,opportunities for radiologic technologist xexx...,flyjofinder
2,New York,2022-02-08 08:06:14+00:00,xfxfxx xfxfxxnnew york rangers coffee mug oz...,topfanscorner
3,New York,2022-02-08 07:56:50+00:00,xfxfxx xfxfxxnultra game na new york knicks m...,topfanscorner
4,New York,2022-02-08 07:55:23+00:00,xfxfxx xfxfxxnteam golf nhl new york rangers ...,topfanscorner


In [7]:
df.describe()

,Location-Search,Date,Tweet,Username
count,121248,121248,121248,121229
unique,13,90585,98005,33188
top,NYC,2022-02-01 07:00:29+00:00,temp xcxc hum dewp xcxc ar hpa rain today ...,ondfireradio
freq,35601,32,67,8391


In [13]:
import nltk

In [14]:
import unicodedata
def tokenization(text):
    text = re.split('\W+', text)
    return text
ps = nltk.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text=text.lower()
    text = re.sub('[0-9]+', '', text)
    text = tokenization(text)
    text=remove_stopwords(text)
    text=stemming(text)
    text=lemmatizer(text)
    return text

df['Tweet'] = df['Tweet'].apply(lambda x: remove_punct(x))
df.head(10)

,Location-Search,Date,Tweet,Username
0,New York,2022-02-08 08:09:30+00:00,"[opportun, mri, technologist, prn, work, vari,...",flyjofinder
1,New York,2022-02-08 08:09:16+00:00,"[opportun, radiolog, technologist, xexx, radio...",flyjofinder
2,New York,2022-02-08 08:06:14+00:00,"[xfxfxx, xfxfxxnnew, york, ranger, coffe, mug,...",topfanscorner
3,New York,2022-02-08 07:56:50+00:00,"[xfxfxx, xfxfxxnultra, game, na, new, york, kn...",topfanscorner
4,New York,2022-02-08 07:55:23+00:00,"[xfxfxx, xfxfxxnteam, golf, nhl, new, york, ra...",topfanscorner
5,New York,2022-02-08 06:12:49+00:00,"[xfxfxx, xfxfxxntervi, made, usa, doul, wall, ...",topfanscorner
6,New York,2022-02-08 06:11:09+00:00,"[riddel, nfl, new, york, giant, full, size, sp...",azcardinalsfa
7,New York,2022-02-08 06:09:47+00:00,"[new, york, citi, council, elect, district, pa...",theigdatastats
8,New York,2022-02-08 06:08:51+00:00,"[lawrenc, taylor, new, york, giant, autograph,...",ltravens
9,New York,2022-02-08 05:56:50+00:00,"[xfxfxx, xfxfxxnamerican, needl, seri, nhl, te...",topfanscorner


In [43]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens]  # remove stopwords and stemming
    return text

In [44]:
#df['Tweet'] = df['Tweet'].apply(lambda x: clean_text(x.lower()))
#df.head()

,Location-Search,Date,Tweet,Username,Location-Tweet
0,New York,2022-02-08 08:09:30+00:00,"[bopportun, for, mri, technologist, prn, work,...",b'FlyJobFinder',"b'Las Vegas, NV'"
1,New York,2022-02-08 08:09:16+00:00,"[bopportun, for, radiolog, technologist, xexx,...",b'FlyJobFinder',"b'Las Vegas, NV'"
2,New York,2022-02-08 08:06:14+00:00,"[bxfxfxx, xfxfxxnnew, york, ranger, coffe, mug...",b'topfanscorner',b''
3,New York,2022-02-08 07:56:50+00:00,"[bxfxfxx, xfxfxxnultra, game, nba, new, york, ...",b'topfanscorner',b''
4,New York,2022-02-08 07:55:23+00:00,"[bxfxfxx, xfxfxxnteam, golf, nhl, new, york, r...",b'topfanscorner',b''


In [15]:
sent = [row for row in df.Tweet]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

['opportun_radiolog',
 'technologist_xexx',
 'radiolog_prn',
 'work_vari',
 'shift',
 'new_york',
 'ny',
 'new_york',
 'applyxexxa_httpstcofktyfcuv']

In [16]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab: 0.09 mins


In [17]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

Time to train the model: 2.34 mins


C:\Users\harsh\AppData\Local\Temp/ipykernel_14160/397523926.py:7: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [18]:
w2v_model.save("word2vec.model")

In [19]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [20]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [21]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('nnyelllowvest_covid', 0.997539758682251),
 ('peak_arif', 0.9975064396858215),
 ('requestnth_sun', 0.997505784034729),
 ('f_fight', 0.9974105358123779),
 ('greec_lemond', 0.997404932975769),
 ('petrok_enpricesnon', 0.9973862171173096),
 ('nwhat_el', 0.9973766803741455),
 ('xfxfxxaexfxfxxxfxfxxcxfxfxxdxfxfxxfxfxfxxaxfxfxxnnpalkisu_nnwionew',
  0.9973551630973816),
 ('xfxfxxdxfxfxxcnncalcutta_kolkata', 0.9973534345626831),
 ('welfar_state', 0.9972971081733704)]

In [26]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [27]:
words = pd.DataFrame(word_vectors.key_to_index.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [28]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [61]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,nyc,"[0.01673779, 0.037369817, -0.047847643, 0.0864...",1,1,1.122588,1.122588
1,london,"[-0.07772713, 0.023762256, 0.0018222457, 0.049...",1,1,1.055934,1.055934
2,amp,"[0.0067200917, 0.10400357, 0.04870397, 0.06199...",0,-1,1.223290,-1.223290
3,kashmir,"[0.020291744, 0.039965585, 0.06920147, 0.10272...",1,1,1.095509,1.095509
4,iran,"[0.07831763, 0.07424875, 0.0368444, -0.0026817...",1,1,1.112111,1.112111
...,...,...,...,...,...,...
42028,no,"[-0.06770401, 0.04623099, 0.022139471, 0.07036...",0,-1,22.716718,-22.716718
42029,superpow,"[-0.06588135, 0.04940139, 0.021077743, 0.06995...",0,-1,17.550976,-17.550976
42030,plane_aort,"[-0.05898397, 0.057558004, -0.022315564, 0.045...",0,-1,2.739327,-2.739327
42031,shittyxexxa,"[-0.09434682, 0.02328472, -0.03703339, 0.03025...",0,-1,2.000369,-2.000369


In [30]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [31]:
sentiment_map = pd.read_csv('sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [37]:
file_weighting = df_orig.copy()

In [47]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text=text.lower()
    text = re.sub('[0-9]+', '', text)
    return text

file_weighting['Tweet']= file_weighting['Tweet'].apply(lambda x: remove_punct(x))

In [64]:
file_weighting

,Location-Search,Date,Tweet,Username,rate
0,New York,2022-02-08 08:09:30+00:00,opportunities for mri technologist prn working...,flyjofinder,1.0
1,New York,2022-02-08 08:09:16+00:00,opportunities for radiologic technologist xexx...,flyjofinder,1.0
2,New York,2022-02-08 08:06:14+00:00,xfxfxx xfxfxxnnew york rangers coffee mug oz...,topfanscorner,1.0
3,New York,2022-02-08 07:56:50+00:00,xfxfxx xfxfxxnultra game na new york knicks m...,topfanscorner,1.0
4,New York,2022-02-08 07:55:23+00:00,xfxfxx xfxfxxnteam golf nhl new york rangers ...,topfanscorner,1.0
...,...,...,...,...,...
121243,NYC,2022-01-31 00:28:12+00:00,revitalize eauty httpstcofuwoymxznnnsanfrancis...,usagoldenoil,1.0
121244,NYC,2022-01-31 00:27:09+00:00,love eing in an nyc ar and the guy next to me ...,travisgrand,1.0
121245,NYC,2022-01-31 00:25:43+00:00,accident two lanes locked in nyconhenryhudson ...,totaltrafficnyc,1.0
121246,NYC,2022-01-31 00:25:15+00:00,listen to ondfire radio station drop y lt head...,ondfireradio,1.0


In [52]:
tfidf = TfidfVectorizer(tokenizer=lambda y:y.split(), norm=None)
tfidf.fit(file_weighting.Tweet)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(file_weighting.Tweet)

C:\Users\harsh\anaconda3\envs\ml\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [53]:
def create_tfidf_dictionary(x, transformed_file, features):
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.Tweet.split()))
 

In [54]:
%%time
replaced_tfidf_scores = (file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1))

Wall time: 50.6 s


In [73]:
def replace_sentiment_words(word, sentiment_dict):
    try:
        out = sentiment_dict[word]
        print(out)
    except KeyError:
        out = 0
    return out

In [97]:
for i in replaced_tfidf_scores:
    i=list(i)

In [72]:
file_weighting

,Location-Search,Date,Tweet,Username,rate
0,New York,2022-02-08 08:09:30+00:00,opportunities for mri technologist prn working...,flyjofinder,0.0
1,New York,2022-02-08 08:09:16+00:00,opportunities for radiologic technologist xexx...,flyjofinder,0.0
2,New York,2022-02-08 08:06:14+00:00,xfxfxx xfxfxxnnew york rangers coffee mug oz...,topfanscorner,0.0
3,New York,2022-02-08 07:56:50+00:00,xfxfxx xfxfxxnultra game na new york knicks m...,topfanscorner,0.0
4,New York,2022-02-08 07:55:23+00:00,xfxfxx xfxfxxnteam golf nhl new york rangers ...,topfanscorner,0.0
...,...,...,...,...,...
121243,NYC,2022-01-31 00:28:12+00:00,revitalize eauty httpstcofuwoymxznnnsanfrancis...,usagoldenoil,0.0
121244,NYC,2022-01-31 00:27:09+00:00,love eing in an nyc ar and the guy next to me ...,travisgrand,0.0
121245,NYC,2022-01-31 00:25:43+00:00,accident two lanes locked in nyconhenryhudson ...,totaltrafficnyc,0.0
121246,NYC,2022-01-31 00:25:15+00:00,listen to ondfire radio station drop y lt head...,ondfireradio,0.0


In [65]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.Tweet, file_weighting.rate]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']

In [66]:
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment
0,"[0, 0, 0, 0, 0, 0, 0, 0, -24.67809917136948, -...","[7.877287824040738, 3.0752359299273624, 11.319...",opportunities for mri technologist prn working...,1.0
1,"[0, 0, 0, 0, 1.3592034755979563, 0, 0, 0, 0, 0...","[7.877287824040738, 3.0752359299273624, 11.606...",opportunities for radiologic technologist xexx...,1.0
2,"[-1.523809890604345, 0, 1.1582759612832656, 0,...","[5.015315540666271, 8.336153708876019, 9.62704...",xfxfxx xfxfxxnnew york rangers coffee mug oz...,1.0
3,"[-1.523809890604345, 0, -2.053054040504245, -3...","[5.015315540666271, 8.1517246697425, 13.753311...",xfxfxx xfxfxxnultra game na new york knicks m...,1.0
4,"[-1.523809890604345, 0, -12.372511065952146, -...","[5.015315540666271, 8.89893907157272, 25.54784...",xfxfxx xfxfxxnteam golf nhl new york rangers ...,1.0
...,...,...,...,...
121243,"[0, 0, 0, 0, 0, 0, 1.1225884521338323, 0, 0]","[11.319307200223149, 7.334963533215376, 11.319...",revitalize eauty httpstcofuwoymxznnnsanfrancis...,1.0
121244,"[1.3354503291869564, 0, -24.67809917136948, -4...","[4.987805350329458, 6.0135178188364105, 2.2708...",love eing in an nyc ar and the guy next to me ...,1.0
121245,"[-15.22044917775636, -1.9997857407074164, 0, 0...","[8.162306779073035, 5.8355481495449135, 9.8152...",accident two lanes locked in nyconhenryhudson ...,1.0
121246,"[1.160417310845451, 0, 0, 1.1889558233340198, ...","[3.6480627796248193, 2.3416873426247724, 7.407...",listen to ondfire radio station drop y lt head...,1.0


list

In [77]:
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x:np.array(x.loc['sentiment_coeff'])@ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df['sentiment'] = replacement_df['prediction']

In [78]:
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[0, 0, 0, 0, 0, 0, 0, 0, -24.67809917136948, -...","[7.877287824040738, 3.0752359299273624, 11.319...",opportunities for mri technologist prn working...,0,-124.651677,0
1,"[0, 0, 0, 0, 1.3592034755979563, 0, 0, 0, 0, 0...","[7.877287824040738, 3.0752359299273624, 11.606...",opportunities for radiologic technologist xexx...,0,-116.919180,0
2,"[-1.523809890604345, 0, 1.1582759612832656, 0,...","[5.015315540666271, 8.336153708876019, 9.62704...",xfxfxx xfxfxxnnew york rangers coffee mug oz...,0,-100.029944,0
3,"[-1.523809890604345, 0, -2.053054040504245, -3...","[5.015315540666271, 8.1517246697425, 13.753311...",xfxfxx xfxfxxnultra game na new york knicks m...,0,-234.253877,0
4,"[-1.523809890604345, 0, -12.372511065952146, -...","[5.015315540666271, 8.89893907157272, 25.54784...",xfxfxx xfxfxxnteam golf nhl new york rangers ...,0,-1100.079226,0
...,...,...,...,...,...,...
121243,"[0, 0, 0, 0, 0, 0, 1.1225884521338323, 0, 0]","[11.319307200223149, 7.334963533215376, 11.319...",revitalize eauty httpstcofuwoymxznnnsanfrancis...,1,3.094222,1
121244,"[1.3354503291869564, 0, -24.67809917136948, -4...","[4.987805350329458, 6.0135178188364105, 2.2708...",love eing in an nyc ar and the guy next to me ...,0,-126.655725,0
121245,"[-15.22044917775636, -1.9997857407074164, 0, 0...","[8.162306779073035, 5.8355481495449135, 9.8152...",accident two lanes locked in nyconhenryhudson ...,0,-431.235739,0
121246,"[1.160417310845451, 0, 0, 1.1889558233340198, ...","[3.6480627796248193, 2.3416873426247724, 7.407...",listen to ondfire radio station drop y lt head...,0,-88.051980,0


In [79]:
predicted_classes = replacement_df.prediction
y_test = replacement_df.sentiment

conf_matrix = pd.DataFrame(confusion_matrix(replacement_df.sentiment, replacement_df.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

Confusion Matrix


,0,1
0,117969,0
1,0,3279



 
 Scores


,scores
accuracy,1.0
precision,1.0
recall,1.0
f1,1.0
